In [1]:
import plotly_express as px
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import mplfinance as mpf
import plotly.graph_objects as go
import yfinance as yf
from backtesting import Backtest, Strategy
import pandas_ta as ta

from backtesting.lib import crossover
import math

In [ ]:
# If you want to check if the strategy is executing trades correctly, use this to validate the data that can be run with the backtest and can handle the plot.
ticker = "usdjpy=X"
df = yf.download(ticker, period="1y", interval="1h")


In [2]:
# For other spreadsheet use this in reading csv
# , delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0

# Darwinex spreadsheet
df = pd.read_csv('../Data/Darwinex/GBPJPY60.csv', header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)


In [3]:
df = df.iloc[:500]

In [53]:
# fig = go.Figure(data=[go.Candlestick(x=df.index,
#                 open=df.Open,
#                 high=df.High,
#                 low=df.Low,
#                 close=df.Close)])

# fig.update_layout(
#     xaxis=dict(type='category', nticks=20),
#     xaxis_rangeslider_visible = False
# )

# fig.show()


In [68]:
# For other spreadsheet use this in reading csv
# , delimiter='\t', names=['Open', 'High', 'Low', 'Close', 'Volume'], header=0

# Darwinex spreadsheet
df = pd.read_csv('../Data/Darwinex/GBPJPY60.csv', header=None, names=['Date', 'Time', 'Open', 'High', 'Low', 'Close', 'Volume'])
df['DateTime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'])
df.set_index('DateTime', inplace=True)
df.drop(columns=['Date', 'Time'], inplace=True)

df = df.iloc[:500]

# Init
conditions = [
    df.Close > df.Open,
    df.Close < df.Open
]

choices = ['Bull', 'Bear']

df['Candle'] = np.select(conditions, choices, default='Doji')
df['Swing_Low'] = None
df['Swing_High'] = None
df['Swing_High'] = None
df['Swing_Low'] = None
df['Trend'] = 'Neutral'

higher_high = float('-inf')
lower_low = float('inf')

bullish_pullback_count = 0
bearish_pullback_count = 0
bullish_pullback = False
bearish_pullback = False

swing_high = df['High'].iloc[0]
swing_low = df['Low'].iloc[0]

uptrend_lowest_pullback = float('inf')
downtrend_highest_pullback = float('inf')

previous_high = df['High'].iloc[0]
previous_low = df['Low'].iloc[0]

current_trend = 'Neutral'
df.at[0, 'Trend'] = current_trend

df['Price_Status'] = None
price_status = None




In [69]:
# Itterating logic starts
for i, row in df.iloc[1:].iterrows():
    current_high = row['High']
    current_low = row['Low']
    candle_type = row['Candle']
    current_close = row['Close']

    if current_close > swing_high:
        current_trend = 'Uptrend'
        swing_low = uptrend_lowest_pullback
        df.at[i, 'Swing_Low'] = swing_low
    elif current_close < swing_low:
        current_trend = 'Downtrend'
    
    df.at[i, 'Trend'] = current_trend

    if bullish_pullback or bearish_pullback:
        price_status = 'Consolidation'
    else:
        price_status = 'Extension'

    if current_trend == 'Uptrend':
        if current_high > previous_high:
            swing_high = current_high
        if current_close < swing_high and candle_type == 'Bear':
            bullish_pullback_count += 1
        if bullish_pullback_count >= 2:
            bullish_pullback = True
        if bullish_pullback:
            if current_low < previous_low and current_close > swing_low:
                uptrend_lowest_pullback = current_low
            if current_close > swing_high:
                bullish_pullback = False
                bullish_pullback_count = 0
                


        

    

In [71]:
df[df['Trend'] == 'Downtrend']

,Open,High,Low,Close,Volume,Candle,Swing_Low,Swing_High,Trend,Price_Status
DateTime,,,,,,,,,,
2013-08-21 11:00:00,152.872,152.991,152.706,152.829,3732.0,Bear,None,None,Downtrend,None
2013-08-21 12:00:00,152.828,152.961,152.683,152.713,3534.0,Bear,None,None,Downtrend,None
2013-08-21 14:00:00,152.958,153.058,152.925,152.952,3292.0,Bear,None,None,Downtrend,None
2013-08-21 16:00:00,153.169,153.321,153.035,153.219,4699.0,Bull,None,None,Downtrend,None
2013-08-21 17:00:00,153.219,153.262,153.028,153.098,3712.0,Bear,None,None,Downtrend,None
...,...,...,...,...,...,...,...,...,...,...
2013-09-19 03:00:00,158.359,158.489,158.318,158.435,2756.0,Bull,None,None,Downtrend,None
2013-09-19 06:00:00,158.693,158.775,158.539,158.601,2398.0,Bear,None,None,Downtrend,None
2013-09-19 07:00:00,158.601,158.644,158.399,158.592,2890.0,Bear,None,None,Downtrend,None


In [72]:
# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=df.index,
                                     open=df['Open'],
                                     high=df['High'],
                                     low=df['Low'],
                                     close=df['Close'],
                                     name='Candlestick')])

# Add swing highs
swing_highs = df[df['Swing_High'].notnull()]
fig.add_trace(go.Scatter(x=swing_highs.index, y=swing_highs['Swing_High'], mode='markers', name='Swing Highs', marker=dict(symbol='triangle-up', color='Green', size=8)))

# Add swing lows
swing_lows = df[df['Swing_Low'].notnull()]
fig.add_trace(go.Scatter(x=swing_lows.index, y=swing_lows['Swing_Low'], mode='markers', name='Swing Lows', marker=dict(symbol='triangle-down', color='Red', size=8)))

# Add trend markers
uptrend_markers = df[df['Trend'] == 'Uptrend']
fig.add_trace(go.Scatter(x=uptrend_markers.index, y=uptrend_markers['Low'] - 0.01 * uptrend_markers['Low'], mode='markers', name='Uptrend', marker=dict(symbol='triangle-up', color='blue', size=8)))

downtrend_markers = df[df['Trend'] == 'Downtrend']
fig.add_trace(go.Scatter(x=downtrend_markers.index, y=downtrend_markers['High'] + 0.01 * downtrend_markers['High'], mode='markers', name='Downtrend', marker=dict(symbol='triangle-down', color='orange', size=8)))

# Update the layout for a clearer view
fig.update_layout(title='Candlestick Chart with Swing Highs and Lows',
                  height=600,
                  xaxis=dict(type='category', nticks=20),
                  xaxis_rangeslider_visible=False)

# Show the figure
fig.show()


In [ ]:
df